In [1]:
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
def log_prog(message):
    with open('code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}')

#### Step 1 --> Extract

In [3]:
def extract(url, table_attribs):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
#extract('https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks')
    table = soup.find('span', string = table_attribs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]\

    log_prog('Data execution completed. Initializing Transfromation progress..')
    return df

#### Step  2--> Transfrom the data

In [4]:
def transform(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)

    log_prog('Data Transformation completed. Initialzing Loading progress..')
    return df

#### Step 3 --> Load the data 

In [5]:
#Loading data to a CSV
def load_into_csv(df, output_path):
    df.to_csv(output_path)
    log_prog('Data succesfully loaded into CSV flie.')

#Loading data to a SQL 
def load_into_sql(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists= 'replace', index=False)
    log_prog('Data Loaded to Database as a table, Executing queries')

def run_query(query_statement, sql_connection):
    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_prog('Process Complete')
    return result

#### Executing Pipeline

In [6]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'

    log_prog('Preliminaries complete. Initiating ETL process')
    df = extract(url, 'By market capitalization')
    transform(df, './input/exchange_rate.csv')
    load_into_csv(df, output_csv_path)
    with sqlite3.connect(database_name) as conn:
        load_into_sql(df, conn, table_name)
        run_query('SELECT * FROM Largest_banks', conn)
        
        run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn)
    
        run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn)
print(df)

   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31    